In [1]:
pip install -r requirements.txt

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import math
import time
import random
import numpy as np
import mindspore as ms 
import mindspore.nn as nn
import mindspore.mint as mint
import matplotlib.pyplot as plt

from download import download
from mingpt.model import GPT
from mingpt.trainer import Trainer
from mingpt.utils import set_seed, print_param_report
from mingpt.bpe import BPETokenizer
from mingpt.lora_inject import inject_lora_into_gpt, attach_lora_optimizer, mark_only_lora_as_trainable
from metrics.perplexity import evaluate_model
from metrics.rouge import eval_rouge
from mindspore.common.initializer import initializer, Normal

[WARNING] ME(2038:281473227190448,MainProcess):2025-10-30-14:31:21.909.010 [mindspore/run_check/_check_version.py:412] Can not find the tbe operator implementation(need by mindspore-ascend). Please check whether the Environment Variable PYTHONPATH is set. For details, refer to the installation guidelines: https://www.mindspore.cn/install
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.10/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.10/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type 

In [3]:
# 华为云端跑的话需要运行这段代码
import os

# 保存原始值（可选，用于后续恢复）
original_rank_table = os.environ.get('RANK_TABLE_FILE')

# 移除环境变量
os.environ.pop('RANK_TABLE_FILE', None)

'/user/config/nbstart_hccl.json'

In [4]:
# 在本地跑可以更换为"CPU"
ms.set_device("Ascend", 0)

[WARNING] ME(2038:281473227190448,MainProcess):2025-10-30-14:31:34.830.454 [mindspore/run_check/_check_version.py:412] Can not find the tbe operator implementation(need by mindspore-ascend). Please check whether the Environment Variable PYTHONPATH is set. For details, refer to the installation guidelines: https://www.mindspore.cn/install


In [5]:
tok = BPETokenizer()

with open("data/input.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

ids = tok(raw_text).asnumpy()[0].astype(np.int32)   # shape: (T,)
print(f"Total BPE tokens: {len(ids):,}")

# 训练集/验证集切分
val_ratio = 0.1
split = int(len(ids) * (1.0 - val_ratio))
train_ids = ids[:split]
val_ids   = ids[split:]

block_size = 1024

class BPEDataset:
    """把一长串token切成很多(x,y)训练样本；自回归：预测下一个token"""
    def __init__(self, arr, block_size):
        self.arr = arr
        self.block = block_size
    def __len__(self):
        # x: [i, i+block-1], y: [i+1, i+block]
        return max(0, len(self.arr) - self.block - 1)
    def __getitem__(self, idx):
        x = self.arr[idx:idx+self.block]
        y = self.arr[idx+1:idx+1+self.block]
        return ms.Tensor(x, ms.int32), ms.Tensor(y, ms.int32)

train_dataset = BPEDataset(train_ids, block_size)
val_dataset = BPEDataset(val_ids, block_size)

Total BPE tokens: 338,025


In [6]:
MODEL_TYPE = 'gpt2'
assert MODEL_TYPE in {'gpt2', 'gpt2-medium', 'gpt2-large'}

# 模型权重下载
url  = f"https://modelers.cn/coderepo/web/v1/file/MindSpore-Lab/minGPT/main/media/{MODEL_TYPE}.ckpt"
CKPT_PATH = f"{MODEL_TYPE}.ckpt"
download(url=url, path=CKPT_PATH, replace=False)
print(f"{MODEL_TYPE} 模型权重下载成功 --> {CKPT_PATH}")

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
gpt2 模型权重下载成功 --> gpt2.ckpt


In [7]:
# 模型实例化
C = GPT.get_default_config()
C.model_type  = MODEL_TYPE
C.vocab_size  = 50257
C.block_size  = 1024
C.embd_pdrop = 0.0
C.resid_pdrop = 0.0
C.attn_pdrop  = 0.0
model_lora = GPT(C)

# 模型权重加载
param_dict = ms.load_checkpoint(CKPT_PATH)
ms.load_param_into_net(model_lora, param_dict)

[WARNING] DEVICE(2038,ffff97b870b0,python):2025-10-30-14:31:38.009.215 [mindspore/ccsrc/plugin/res_manager/ascend/mem_manager/ascend_vmm_adapter.h:150] CheckVmmDriverVersion] Open file /etc/ascend_install.info failed.
[WARNING] DEVICE(2038,ffff97b870b0,python):2025-10-30-14:31:38.009.308 [mindspore/ccsrc/plugin/res_manager/ascend/mem_manager/ascend_vmm_adapter.h:189] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6


number of parameters: 124.44M


([],
 ['h.0.attn.bias',
  'h.1.attn.bias',
  'h.2.attn.bias',
  'h.3.attn.bias',
  'h.4.attn.bias',
  'h.5.attn.bias',
  'h.6.attn.bias',
  'h.7.attn.bias',
  'h.8.attn.bias',
  'h.9.attn.bias',
  'h.10.attn.bias',
  'h.11.attn.bias'])

In [8]:
# 加入LoRA层
inject_lora_into_gpt(
    model_lora,
    r=8,
    alpha=8.0,
    dropout=0.0,
    target=("qkv", "attn_out", "mlp"),
    include_lm_head=False
)

# 只训 LoRA
mark_only_lora_as_trainable(model_lora)
attach_lora_optimizer(model_lora)

In [9]:
# 训练配置（LoRA参数微调）
T_lora = Trainer.get_default_config()
T_lora.max_iters      = 10
T_lora.batch_size     = 8
T_lora.learning_rate  = 1e-3
T_lora.weight_decay   = 0.0
T_lora.num_workers    = 2

trainer_lora = Trainer(T_lora, model_lora, train_dataset)

iters_lora, losses_lora = [], []

# 日志输出（每100步）+保存权重（每500步）
os.makedirs("out-lora", exist_ok=True)

def cb(tr):
    if tr.iter_num % 1 == 0:
        batch_size = tr.config.batch_size
        # 打印训练损失，训练时长和每次批次的数据量
        print(f"iter:{tr.iter_num:5d} | loss:{tr.loss.asnumpy():.4f} | dt:{tr.iter_dt*1000:.1f}ms | batch size: {batch_size}")
        iters_lora.append(tr.iter_num)
        losses_lora.append(float(tr.loss.asnumpy()))

    if tr.iter_num % 500 == 0 and tr.iter_num > 0:
        path = f"out-lora/lora_gpt2_shakespeare_step{tr.iter_num}.ckpt"
        ms.save_checkpoint(model_lora, path)
        print(f"Saved step ckpt -> {path}")
        
trainer_lora.add_callback('on_batch_end', cb)

In [10]:
t0_lora = time.time()
# 训练（LoRA微调）
trainer_lora.run()

final_ckpt = "out-lora/lora_gpt2_shakespeare_final.ckpt" 
ms.save_checkpoint(model_lora, final_ckpt)
print(f"Saved final ckpt -> {final_ckpt}")

t_lora = time.time() - t0_lora

(Parameter (name=c_attn.lora_0.A, shape=(8, 768), dtype=Float32, requires_grad=True), Parameter (name=c_attn.lora_0.B, shape=(2304, 8), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_1.A, shape=(8, 768), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_1.B, shape=(768, 8), dtype=Float32, requires_grad=True), Parameter (name=c_fc.lora_2.A, shape=(8, 768), dtype=Float32, requires_grad=True), Parameter (name=c_fc.lora_2.B, shape=(3072, 8), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_3.A, shape=(8, 3072), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_3.B, shape=(768, 8), dtype=Float32, requires_grad=True), Parameter (name=c_attn.lora_4.A, shape=(8, 768), dtype=Float32, requires_grad=True), Parameter (name=c_attn.lora_4.B, shape=(2304, 8), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_5.A, shape=(8, 768), dtype=Float32, requires_grad=True), Parameter (name=c_proj.lora_5.B, shape=(768, 8), dtype=Float32, 